## INTRODUCTION

Inspired by the wave analyses and manipulations that homework 9 required, as well as by our own experiences as musicians, we chose to create what we are calling an autosampler. In a musical context, sampling is the act of selecting and isolating small parts (known as samples) of a recording, and using those samples as part of a new piece of music. Sampling has always been common in the world of hip hop, where drum beat samples are looped and used as a backing track. More recently, some producers of electronic dance music have risen to prominence whose music consists almost entirely of short samples, mixed together and played as their own instrument. In either case, the producer needed to manually create his own samples. Using modern-day software, this is done by simply selecting a starting and an ending point on a recording's sound wave, and taking the part of the wave that lies between those two points. While this process sounds simple enough, it becomes quite tedious when you have large numbers of recordings to analyze, listen to for potential samples, and cut samples from. Our goal in this project is to automatically cut samples from user-provided wave files, and arrange those samples to produce output in the tune of a user-provided melody. For the examples presented in this paper, we have used the melody of Beethoven's well known piece Fur Elise.

## CODE DESCRIPTION:

The meat of this project is the functionality of chopping up an input wave into samples, and then properly categorizing and storing the resulting pieces. In our initial, naive approach, we decided to chop the waves such that one sample would have the length of an eighth note, given a predefined tempo in beats per minute. We arbitrarily chose a tempo of 148 beats per minute. This translated to approximately 2.467 beats per second, or 0.2027 seconds per note since the eighth note is traditionally half a beat. Using a standard sampling rate of 44.1 kHz, this meant that each note would be 8939 data points long.

With this information, the next step is to iterate over each input wave provided by the user. Using the functions from waveIO.py, each file is read in and unpacked.The wave is then split, as mentioned above, into pieces 8939 data points long. This list of points serves as the samples that will be analyzed, sorted, and built into a final output wave.

This list of samples is then iterated over for sorting and storage. Samples shorter than 8939 samples long are discarded, which prevents shortened samples such as those that can occur at the end of a wave from being sorted with the samples of uniform length. The Fourier Transform of the sample is taken using Numpy's fft module, and then divided by the length of the sample in data points to normalize its amplitudes. Numpy'sfftfreq function is used to create the frequency bins for the sample, and the sample's strongest frequency is found.

This is where the amplitude normalization becomes important. The amplitude of the strongest frequency is compared to a predefined threshold value. If the amplitude is lower than this threshold, then no frequencies are extracted from the sample. If the amplitude is greater than the threshold, the sample's data is added to a large list of samples, and its index within that list is noted. The sample proceeds with sorting. In that case, the frequency is compared with a dictionary containing baseline frequencies for the twelve semitones in a musical octave. Since the A4 (this notation stands for the note A in the fourth octave, with A being the tenth not in the octave) note with an ideal frequency of 440.0 Hz is typically used as a standard, I used the frequencies of all the fourth octave notes to populate this dictionary.

If the sample's frequency is lower than the lowest frequency in the dictionary (with about 1% tolerance), then it must be part of a lower octave. Likewise, if it is higher than the highest frequency in the dictionary (with the same tolerance), it must be part of a higher octave. In the former case, I divide a multiplier value (initially set to 1) by 2 and compare the sample frequency to the highest and lowest frequencies in the dictionary, multiplied by the multiplier value. If the frequency is still too low, I again halve the multiplier, continuing this process until the sample frequency falls between the highest and lowest dictionary frequencies times the multiplier. Alternatively, in the case that the sample frequency was too high, I perform the same process, but double the multiplier instead of halving it on each unsuccessful comparison.

Having ascertained the octave of the sample frequency, and with specific values of the multiplier corresponding to specific octaves, the sample can be properly stored. The dictionary of note frequencies is iterated over, with each note's frequency value multiplied by the multiplier from earlier. If the sample frequency is within 1% of this multiplied dictionary value, its index in the larger sample list is stored in a list within a dictionary of the following structure:

In [1]:
notes_db = {
"A": [[],[],[],[],[],[]],
"A#": [[],[],[],[],[],[]],
"B": [[],[],[],[],[],[]],
"C": [[],[],[],[],[],[]],
"C#": [[],[],[],[],[],[]],
"D": [[],[],[],[],[],[]],
"D#": [[],[],[],[],[],[]],
"E": [[],[],[],[],[],[]],
"F": [[],[],[],[],[],[]],
"F#": [[],[],[],[],[],[]],
"G": [[],[],[],[],[],[]],
"G#":[[],[],[],[],[],[]]
}


Where each note has a list of sub-lists, and each sub-list holds the index values of samples that contain the corresponding note in a specific octave.

If the code were to stop at this point, it could deal perfectly fine with simple input waves of single notes. Chords, however, would pose a problem, as only one frequency is pulled from each sample. To solve this problem, after the sample is sorted into one of the bins above, the strongest frequency is deleted from the Fourier Transform data, as is the frequency bin that corresponded to that frequency. The octave multiplier is reset to 1, and the next strongest frequency is pulled from the Fourier data. Its amplitude is once again compared to the threshold, and the whole process repeats itself, selecting, sorting, and deleting the strongest frequencies from the sample until the strongest frequency is lower than the threshold. Once this happens, the sample is completely sorted, and the sample's index is stored in bins for each note that was contained within the sample. It is important to note that on these subsequent loops, a frequency will only be added to a bin if the same sample's index does not already appear in that bin. This prevents a sample from being sorted twice into the same note's bin.

For example, after completely sorting a sample of a C Major chord in the fourth octave, where all three notes were above threshold and there was no noise above the threshold, the notes_db structure would look like this:

In [ ]:
{
"A": [[],[],[],[],[],[]],
"A#": [[],[],[],[],[],[]],
"B": [[],[],[],[],[],[]],
"C": [[],[],[],[sample_index],[],[]],
"C#": [[],[],[],[],[],[]],
"D": [[],[],[],[],[],[]],
"D#": [[],[],[],[],[],[]],
"E": [[],[],[],[sample_index],[],[]],
"F": [[],[],[],[],[],[]],
"F#": [[],[],[],[],[],[]],
"G": [[],[],[],[sample_index],[],[]],
"G#":[[],[],[],[],[],[]]
}

Once all the samples from a given input wave are sorted in this manner, it is time to assemble them into the music the user provided. A sample of the notation used to specify the output music is shown below.

This example consists of the first few measures of Fur Elise. The notes to be played at a given time appear on each line. "#" symbols are used to denote boundaries between measures for convenience of reading and writing, and are ignored by the program. "%" symbols denote rests, or periods where no notes are played. Each note or rest has the length of one sample. Chords can be denoted by having multiple notes on the same line, separated by commas.

First an empty array is initialized to hold the output wave. Each line of the input music file is isolated and split by comma characters. If the line consists of a "#" symbol, the line is ignored and the next line is processed. If the line consists of a "%" symbol, a sample-length set of zeros is generated and appended to the output wave. Otherwise, for each note symbol on the line, a random sample index is pulled from the note's corresponding bin. The sample whose index was chosen is appended onto the output wave.

Once all the lines have been processed, the output wave is packed and saved under a user-provided filename, using functions from waveIO.

## Physical Description

In the realm of soundwave analysis, there are a number of "perspectives" from which a wave can be viewed. The first of these, and likely the most familiar to many people, is the time domain perspective. The time domain perspective for a given soundwave shows the wave's amplitude over time. In music production, this perspective is often used to visually identify buildups, musical climaxes, and changes from one section of a song to another. It is this perspective which is typically used when a visual representation of a sound wave is needed.

<img src="Results/figures/vocal_scales_time_domain.png">[Fig 1]

The figure above is an example of the time perspective. From this perspective, it is easily seen that there are four distinct sections, each separated by short segments of much lower volume. However, little information about the contents of those louder sections can be gathered from this view.

One aspect of the wave which this view gives no information on, but which is very important for musical application is frequency. The frequency of a wave determines the pitch of the sound a wave makes. The higher the wave's frequency, the higher pitched its note will sound. Musical notes are organized into sets called octaves. An octave consists of twelve different frequencies, called semitones, that represent musical notes. The lowest note in an octave is C, the highest is B. The standard notation "C4" stands for the C note in the fourth octave, which has a frequency of 261.63 Hz. To find the frequency of a note *n* semitones higher than frequency *f*, the following formula is used:

<b><i>f_new = f \* 2<sup>n/12</sup></i></b>

From this formula, the conclusion can be drawn that a note exactly one octave, or twelve semitones, above frequency *f* has the frequency *2f*:

<b><i>f_octave = f \* 2<sup>12/12</sup> = f \* 2 = 2f</i></b>

With this knowledge, the next logical perspective to look at is the frequency perspective. This perspective is used to determine what frequencies are present in a given wave. To obtain this perspective, the Fourier Transform is performed on the wave. With the resulting amplitudes normalized by dividing them by the number of samples in the wave, the result looks like this:

<img src="Results/figures/sample_wave_ps.png">[Fig 2]

This example shows the frequency perspective of a simple, computer generated wave. Clearly there are three main frequencies present. They are at approximately 440 Hz, 550 Hz, and 660 Hz. The wave used to generate this spectrum consisted of short segments where each of those frequencies (corresponding to A4, C#5, and E5, respectively, the three notes in an A Major chord) is played individually, then a final segment where the three were played together. A second sample wave was generated in which the same notes were played in reverse order, before again finishing with all three played together. Observe the frequency domain of this sample:

<img src="Results/figures/sample_wave_ps_reverse.png">[Fig 3]

The two plots look identical! This illustrates an important aspect of the frequency domain: just as the time domain conveyed no usable information about the frequencies contained within its wave, the frequency domain conveys no information about the times at which its frequencies occur.

Ideally, there would be a perspective that can show both frequency and time simultaneously. This way, we could isolate exactly what notes are happening and when. Such a perspective would then allow for hand picking sections of the wave to take samples from, if manual sampling were the goal. As it happens, there is a perspective that does exactly this: the spectrogram. A spectrogram is a plot of frequency strength over time. The horizontal axis typically represents time, while the vertical represents various frequencies. to denote amplitude of a frequency at a given time, the corresponding point on the plot is colored. In the following examples, dark red indicates high amplitude at the frequency and time, and yellow indicates low amplitude. The following are spectrograms provided by matplotlib for the two sample waves used for the power spectra above:

<img src="Results/figures/sample_wave_specgram.png">[Fig 4]

<img src="Results/figures/sample_wave_specgram_reverse.png">[Fig 5]

In these spectrograms, the red bands center around the frequencies being played at a given time in the wave. It is clear, looking at them, that the two samples contain the same notes, being played in a different order, and it is clear exactly what that order is for each sample.

The spectrograms above were generated easily from wave data using matplotlib's `specgram` function, however the method by which the function generates them is the key to this project. `Specgram` makes use of what is known as the **Short Time Fourier Transform**, or STFT. The STFT is defined as follows:

<img src="STFT equation.png">[Fig 6, Bebis]

STFT essentially takes the Fourier Transform of very small parts of the wave at a time. This is done by multiplying the wave by a windowing function, which is nonzero only in a small time range centered around time `t'`, and then taking the Fourier Transform of the result. These small transforms are taken over the entire length of the wave by incrementally increasing `t'`, and a matrix can be created showing the intensity of all frequencies in all the small parts of the wave. These matrices can be plotted as the spectrograms seen above.

Looking again at the spectrograms, there is one key difference other than simply the order in which the notes are presented. The thicknesses of the frequency bars are different in the two figures. This is a factor of how large the STFT window is. A longer STFT window provides greater frequency precision, showing more clearly exactly what frequencies are represented. There is a tradeoff here, though. Imagine an STFT whose window width was equal to the length of the entire wave. This would be the same as a normal Fourier Transform! That is to say we would lose all indication of the frequencies' timing! The frequency resolution given by STFT with a given window length L is defined by:

$\Delta F = F_{s} / L Hz$

Conversely, imagine an STFT with an infinitely short windowing function such as the Dirac Delta Function:

<img src="STFT infinite short.png">[Fig 7, Bebis]

As you can see, because this windowing function gives a result of zero for all points other than `t'`, the result of the STFT would just be a time spectrum of the wave, losing all frequency representation (Bebis).


For the purposes of this project, I am not trying to build a spectrogram, however, but rather take apart the wave into small pieces (samples), which can then be put together in any order to create a new piece of choesive music. The principle, however, remains the same.


**Results**

I began by building a naive version of the STFT (`sampler-naive.py`). This chopped up the input waves into equally sized chunks (in this case 8939 data points long), and then computed the Fourier Transform on each chunk to give myself a set of equally sized samples with which to build my output wave. This is equivalent to performing an STFT with window size 8939, and $\Delta_{t'} = 4469$. What this means is that there is no overlap between the STFT windows; the next window begins where the previous window ended. I then created a wave file which consisted of a chromatic scale from C4 to G#6. This wave was called `bigsample.wav`, Every note in that range was represented, which made it ideal for sampling and putting together into a new piece.

I ran `bigsample.wav` through my naive function, which chopped it up successfully and rearranged the samples into `furelise-naive.wav`. I also plotted how many samples per note between C4 and G#6 the function was able to get from the wave. That plot is below:

<img src="Results/figures/note_numbers_naive.png">[Fig 8]

There are at least two samples per note, with some notes having three. The reason some notes have an extra sample has to do with sample windows which lie on boundaries between two notes playing. These boundaries would count as a sample for both of the two notes contained within the window. Listening to `furelise-naive.wav` confirms this. Every so often a sample is heard that sounds like two notes, each a fraction of the normal note length. These are those boundary-case samples. Comparing the spectrogram of `furelise-naive.wav` to that of an analytically generated `analytic_furelise.wav` files, we can see these boundary cases causing some discrepancies, namely around the 2.25 second mark: 

<img src = "Results/figures/analytic_furelise_specgram.png">[Fig 9]

<img src = "Results/figures/naive_furelise_specgram.png">[Fig 10]

With a better understanding of STFT, I restructured my naive code to make manipulating the parameters of the STFT more manageable. The result was `sampler-stft.py`. To ensure that it was working as desired, I set it up with the same initial conditions as my naive approach had used: window size set to 8939, window step set to the window size, and using the same `bigsample.wav` as the input wave. The plot of samples per note is identical to that generated by my naive model:

<img src="Results/figures/note_numbers_stft_noverlap.png">[Fig 11]

To reduce the boundary cases that make it into the output wave, it makes sense to shorten the window length. While the number of boundary cases is likely to stay the same or even increase slightly (There are only so many note boundaries in the wave to begin with), the number ofsamples pulled from the file will increase more so that the boundary cases are less likely to be selected. Using a  window length of 4500, the samples per note plot changes in a couple notable ways:

<img src="Results/figures/note_numbers_smallwindow.png">[Fig 12]

As expected the number of samples has roughly doubled. Most notes now have 4 samples instead of two, with some notes having five. More importantly, though, E4 has no samples! There are two factors at play here: frequency resolution, and tolerance in my note sorting algorithm. By reducing my window size, I have also reduced my frequency resolution from approximately 4.9 Hz to 9.8 Hz. With this lower frequency resolution, I need to raise my tolerance to be less restrictive when sorting notes.

**Bibliography:**

Bebis. Short Time Fourier Transform (STFT). Retrieved from University of Nevada, Reno Computer Science and Engineering department, via Google: www.cse.unr.edu%2F~bebis%2FCS474%2FLectures%2FShortTimeFourierTransform.ppt

Heinzel, Rüdiger, Schilling. "Spectrum and Spectral density estimation by the Discrete Fourier Transform (DFT), including a comprehensive list of window functions and some new flat-top windows." February 15, 2002. Max-Planck-Institute für Gravitationsphysik. December 8, 2015 <https://holometer.fnal.gov/GH_FFT.pdf>

"Piano Key Frequencies." Wikipedia. December 8, 2015. <https://en.wikipedia.org/wiki/Piano_key_frequencies>